In [1]:
import os
import queue
import threading
import time

import pandas
from lxml import html
from selenium.webdriver import Chrome, Edge
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager


def GetChromeBrowser(isHeadless=True):
    opt = Options()
    if isHeadless:
        opt.add_argument("--headless")
    opt.add_argument("--mute-audio")
    opt.add_argument("--disable-notifications")
    capabilities = DesiredCapabilities.CHROME
    capabilities["goog:loggingPrefs"] = {"performance": "ALL"}
    try:
        return Chrome(service=Service(ChromeDriverManager().install()), options=opt)
    except Exception as err:
        print(err)


def GetEdgeBrowser(isHeadless=True):
    options = EdgeOptions()
    if isHeadless:
        options.add_argument("--headless")
    options.add_argument("--mute-audio")
    options.add_argument("--disable-notifications")

    try:
        return Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
    except Exception as err:
        print(err)


resultQ = queue.Queue()
errorQ = queue.Queue()


def Scrapper(driver, url):
    driver.get(url)

    curProductXpath = '//div[@class="product-right"]'
    titleXpath = 'descendant::span[@data-ui-id="page-title-wrapper"]/text()'
    priceXpath = 'descendant::span[@class="price"]/text()'
    skuXpath = 'descendant::div[@itemprop="sku"]/text()'
    descXpath = 'descendant::*[@class="product attribute description"]//text()'
    availabilityXpath = 'descendant::div[@title="Availability"]/descendant::text()'

    moreHeaderXpath = 'descendant::*[@class="Specifications-txt row"]//div[@class="Stxt"]/child::*[1]/text()'
    moreValXpath = 'descendant::*[@class="Specifications-txt row"]//div[@class="Stxt"]/child::*[2]/text()'

    images = '//div[@class="fotorama__stage__shaft fotorama__grab"]//descendant::img/@src'

    try:
        cc_element = WebDriverWait(driver, 25).until(
            EC.visibility_of_all_elements_located((By.XPATH, images)))
    except:
        pass

    time.sleep(1)

    page_source = driver.page_source
    doc = html.fromstring(page_source)
    d_data = {}
    right_product = doc.xpath(curProductXpath)[0]
    d_data['title'] = '\n'.join(right_product.xpath(titleXpath))
    d_data['price'] = '\n'.join(right_product.xpath(priceXpath))
    d_data['description'] = '\n'.join(right_product.xpath(descXpath))
    d_data['sku'] = '\n'.join(right_product.xpath(skuXpath))
    d_data['availability'] = '\n'.join(right_product.xpath(availabilityXpath))
    header = right_product.xpath(moreHeaderXpath)
    values = right_product.xpath(moreValXpath)

    for indx, head in enumerate(header):
        d_data[head] = values[indx]

    d_data['images'] = '\n'.join(doc.xpath(images))
    d_data['product'] = url
    resultQ.put(d_data)


files = os.listdir('myntra_men')

jobs = queue.Queue()


def WorkerLoop(progress_bar: tqdm):
    # driver = GetChromeBrowser(False)
    driver = GetEdgeBrowser(False)
    while not jobs.empty():
        url = jobs.get()
        try:
            Scrapper(driver, url)
        except:
            errorQ.put({'url': url})

            pass
        time.sleep(1)
        progress_bar.update()


i = 'kankatala  BANARASI.xlsx'
path = f'kalamandir_files/{i}'

if i.__contains__('.csv'):
    data = pandas.read_csv(path)
else:
    data = pandas.read_excel(path)

product_column = 'product href'
urls = list(data[product_column])

[jobs.put(_) for _ in urls]
threads = []
pbar = tqdm(total=len(urls))

for worker in range(6):
    thread = threading.Thread(target=WorkerLoop, args=(pbar,))
    thread.start()
    threads.append(thread)

for t in threads:
    t.join()


def qToDf(q):
    l = []
    while not q.empty():
        l.append(q.get())
    [q.put(_) for _ in l]
    df = pandas.DataFrame(l)
    return df


qToDf(resultQ).to_csv(f'{"".join(i.split(".")[:-1])}_scrapped_result.csv', index=False)
qToDf(errorQ).to_csv(f'{"".join(i.split(".")[:-1])}_scrapped_error.csv', index=False)



100%|██████████| 169/169 [02:15<00:00,  6.18it/s]